# Introducing Distributed Word Vectors

IMDB 영화 리뷰 데이터로 

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [10]:
train.tail(5)

,id,sentiment,review
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."
24999,"""8478_8""",1,"""I saw this movie as a child and it broke my h..."


In [11]:
test.tail(5)

,id,review
24995,"""2155_10""","""Sony Pictures Classics, I'm looking at you! S..."
24996,"""59_10""","""I always felt that Ms. Merkerson had never go..."
24997,"""2531_1""","""I was so disappointed in this movie. I am ver..."
24998,"""7772_8""","""From the opening sequence, filled with black ..."
24999,"""11465_10""","""This is a great horror film for people who do..."


In [12]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, "lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [13]:
# Download the punkt tokenizer for sentence splitting
#import nltk.data
#nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

<img src="nltk.png" width="500"><BR>

In [14]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


In [15]:
print(len(sentences))

795538


In [16]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


# Training and Saving Your Model


In [18]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-06-08 13:42:08,737 : INFO : 'pattern' package not found; tag filters are not available for English
2018-06-08 13:42:08,745 : INFO : collecting all words and their counts
2018-06-08 13:42:08,745 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-08 13:42:08,801 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-06-08 13:42:08,857 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-06-08 13:42:08,909 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2018-06-08 13:42:08,963 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-06-08 13:42:09,017 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-06-08 13:42:09,066 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-06-08 13:42:09,114 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-06-08 13:42:09,158 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-06-08 13:42:09,210 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-06-08 13:42:09,258 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-06-08 13:42:09,305 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2018-06-08 13:42:09,351 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2018-06-08 13:42:13,282 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-06-08 13:42:13,341 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-06-08 13:42:13,447 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-06-08 13:42:13,531 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-06-08 13:42:13,580 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-06-08 13:42:13,581 : INFO : Loading a fresh vocabulary
2018-06-08 13:42:13,685 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-06-08 13:42:13,686 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-06-08 13:42:13,760 : INFO : deleting the raw counts dictionary of 123504 items
2018-06-08 13:42:13,767 : INFO : sample=0.001 d

2018-06-08 13:43:02,536 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 13:43:02,547 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 13:43:02,554 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 13:43:02,561 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 13:43:02,562 : INFO : EPOCH - 4 : training on 17798082 raw words (12749727 effective words) took 11.9s, 1075307 effective words/s
2018-06-08 13:43:03,570 : INFO : EPOCH 5 - PROGRESS: at 8.28% examples, 1050148 words/s, in_qsize 7, out_qsize 0
2018-06-08 13:43:04,574 : INFO : EPOCH 5 - PROGRESS: at 16.84% examples, 1063191 words/s, in_qsize 7, out_qsize 0
2018-06-08 13:43:05,576 : INFO : EPOCH 5 - PROGRESS: at 25.33% examples, 1067619 words/s, in_qsize 7, out_qsize 0
2018-06-08 13:43:06,578 : INFO : EPOCH 5 - PROGRESS: at 33.88% examples, 1069944 words/s, in_qsize 7, out_qsize 0
2018-06-08 13:43:07,579 : INFO : EPOCH 5

# Exploring the Model Results

In [19]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [20]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [21]:
model.doesnt_match("paris berlin london austria".split())

'paris'

In [22]:
model.most_similar("man")

[('woman', 0.6268669366836548),
 ('lady', 0.5913753509521484),
 ('lad', 0.5675976276397705),
 ('guy', 0.5300781726837158),
 ('person', 0.5196842551231384),
 ('monk', 0.5194322466850281),
 ('farmer', 0.5101560950279236),
 ('sailor', 0.5071306824684143),
 ('soldier', 0.5039519667625427),
 ('men', 0.502257227897644)]

In [23]:
model.most_similar("queen")

[('princess', 0.648817777633667),
 ('belle', 0.6002864241600037),
 ('eva', 0.599858283996582),
 ('mistress', 0.5996459126472473),
 ('maid', 0.5906264781951904),
 ('regina', 0.5899233818054199),
 ('victoria', 0.5890839099884033),
 ('bride', 0.5841931104660034),
 ('latifah', 0.583801805973053),
 ('maria', 0.581763505935669)]

In [24]:
model.most_similar("awful")

[('terrible', 0.77015221118927),
 ('horrible', 0.7469659447669983),
 ('atrocious', 0.7335578203201294),
 ('abysmal', 0.7048677802085876),
 ('dreadful', 0.7017945647239685),
 ('appalling', 0.6771308779716492),
 ('horrid', 0.6641309261322021),
 ('horrendous', 0.6565769910812378),
 ('lousy', 0.6234528422355652),
 ('laughable', 0.6173096895217896)]

# More Fun With Word Vectors

## Numeric Representations of Words

In [25]:
from gensim.models import Word2Vec

In [26]:
model = Word2Vec.load("300features_40minwords_10context")

2018-06-08 13:47:14,180 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-06-08 13:47:14,520 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-06-08 13:47:14,521 : INFO : setting ignored attribute vectors_norm to None
2018-06-08 13:47:14,521 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-06-08 13:47:14,522 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-06-08 13:47:14,522 : INFO : setting ignored attribute cum_table to None
2018-06-08 13:47:14,523 : INFO : loaded 300features_40minwords_10context


In [28]:
type(model.wv.syn0)

numpy.ndarray

In [29]:
model.wv.syn0.shape

(16490, 300)

In [39]:
model.wv['flower'].shape

(300,)

In [31]:
model["flower"]

array([-0.02126038, -0.07174639, -0.09453137,  0.02006711, -0.00061063,
        0.04208945,  0.06831188, -0.02231076,  0.07070734, -0.06267736,
       -0.04803132,  0.05455393,  0.053824  , -0.01698873,  0.08780768,
        0.03812271,  0.05156148,  0.00542303,  0.12575129,  0.02209862,
        0.05048895, -0.06192749,  0.01876858, -0.07182054,  0.0736201 ,
        0.01810209,  0.11593132, -0.02639383,  0.00668718,  0.07856153,
        0.0070363 ,  0.06570432,  0.06874821,  0.02721597,  0.01020559,
       -0.03201872, -0.04692639,  0.04517887,  0.00139306, -0.011076  ,
       -0.03086644,  0.09124836,  0.06479169,  0.01176843, -0.03789015,
       -0.01299938,  0.00196332, -0.01648852, -0.00323442,  0.08011638,
        0.09551287, -0.0385311 , -0.02601301, -0.0650504 ,  0.03706691,
        0.1123137 , -0.00666139,  0.00765615,  0.02689798, -0.11085436,
       -0.04756327,  0.0966139 ,  0.12006098, -0.04014163,  0.17065212,
        0.07642167,  0.03194519, -0.05441636, -0.0714246 , -0.07

# From Words To Paragraphs, Attempt 1: Vector Averaging

In [67]:
import numpy as np

In [68]:
def makeFeatureVec(words, model, num_features):

    # Pre-initialize an empty numpy array for speed
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords+1.
            featureVec = np.add(featureVec, model[word])
    
    # Divide the result by the number of words to get the avg
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [69]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    counter = int(counter)
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        
       # Call the function (defined above) that makes average feature vectors, increment the counter
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
        
    return reviewFeatureVecs

In [70]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

print ("Creating average feature vecs for Training reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Creating average feature vecs for Training reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [71]:
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [72]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
trainDataVecs = Imputer().fit_transform(trainDataVecs)

forest = forest.fit( trainDataVecs, train["sentiment"] )

Fitting a random forest to labeled training data...


In [73]:
# Test & extract results 
testDataVecs = Imputer().fit_transform(testDataVecs)
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [74]:
from sklearn.cluster import KMeans
import time
start = time.time() # Start time

In [75]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors
num_clusters = int(word_vectors.shape[0] / 5)

In [76]:
# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

In [77]:
# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  456.84869408607483 seconds.


In [78]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number 
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [79]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map)[i])
    print(words)


Cluster 0
['undeserved']

Cluster 1
['picard', 'sisko']

Cluster 2
['groovy', 'rocking']

Cluster 3
['contradictions']

Cluster 4
['logo', 'buses']

Cluster 5
['terrorist', 'unit', 'squad', 'traitor', 'bomber', 'swat', 'samir', 'ripley']

Cluster 6
['introduced', 'treated', 'witness', 'reduced', 'subjected', 'accustomed']

Cluster 7
['endearing', 'eccentric', 'enigmatic', 'oddball', 'amiable', 'astute', 'affectionate', 'unsavory', 'affable', 'loveable']

Cluster 8
['turner', 'ross', 'irving', 'topper', 'burnett', 'holliday', 'guild', 'keeler']

Cluster 9
['clubs', 'restaurants', 'hotels']


In [80]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [81]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [82]:
# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [83]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


python3 code 

https://github.com/tneupaney/Bag-of-Words-Meets-Bags-of-Popcorn